In [1]:
import Pkg;
Pkg.activate(@__DIR__)
Pkg.status()

  Activating project at `/global/u1/b/blaschke/juliacon24-hpcworkshop/parts/mpi/explanation`


Status `/global/u1/b/blaschke/juliacon24-hpcworkshop/parts/mpi/explanation/Project.toml`
  [1520ce14] AbstractTrees v0.4.5
  [052768ef] CUDA v5.4.2
  [adafc99b] CpuId v0.3.1
  [0e44f5e4] Hwloc v3.0.1
  [da04e1cc] MPI v0.20.20
  [e7922434] MPIClusterManagers v0.2.4
  [6f74fd91] NetworkInterfaceControllers v0.1.0


In [2]:
using CpuId

const cpucycle_mask = (
    (1 << (64 - leading_zeros(CpuId.cputhreads()))) - 1
) % UInt32

cpucycle_coreid() = Int(cpucycle_id()[2] & cpucycle_mask)

cpucycle_coreid (generic function with 1 method)

In [3]:
cpucycle_coreid()

13

In [4]:
using MPI, CUDA

In [5]:
MPI.versioninfo()

MPIPreferences:
  binary:  system
  abi:     MPICH
  libmpi:  libmpi_gnu_123.so
  mpiexec: srun

Package versions
  MPI.jl:             0.20.20
  MPIPreferences.jl:  0.1.11

Library information:
  libmpi:  libmpi_gnu_123.so
  libmpi dlpath:  /opt/cray/pe/lib64/libmpi_gnu_123.so
  MPI version:  3.1.0
  Library version:  
    MPI VERSION    : CRAY MPICH version 8.1.28.29 (ANL base 3.4a2)
    MPI BUILD INFO : Wed Nov 15 20:57 2023 (git hash 1cde46f)
    


In [6]:
CUDA.versioninfo()

CUDA runtime 12.2, local installation
CUDA driver 12.2
NVIDIA driver 525.105.17

CUDA libraries: 
- CUBLAS: 12.2.1
- CURAND: 10.3.3
- CUFFT: 11.0.8
- CUSOLVER: 11.5.0
- CUSPARSE: 12.1.1
- CUPTI: 20.0.0
- NVML: 12.0.0+525.105.17

Julia packages: 
- CUDA: 5.4.2
- CUDA_Driver_jll: 0.9.1+1
- CUDA_Runtime_jll: 0.14.1+0
- CUDA_Runtime_Discovery: 0.3.4

Toolchain:
- Julia: 1.9.4
- LLVM: 14.0.6

Preferences:
- CUDA_Runtime_jll.version: 12.2
- CUDA_Runtime_jll.local: true

4 devices:
  0: NVIDIA A100-SXM4-80GB (sm_80, 79.150 GiB / 80.000 GiB available)
  1: NVIDIA A100-SXM4-80GB (sm_80, 79.150 GiB / 80.000 GiB available)
  2: NVIDIA A100-SXM4-80GB (sm_80, 79.150 GiB / 80.000 GiB available)
  3: NVIDIA A100-SXM4-80GB (sm_80, 79.150 GiB / 80.000 GiB available)


In [7]:
import Base: filter, Fix1
filter(f::Function)::Function = Fix1(filter, f)

filter (generic function with 26 methods)

In [8]:
function get_device_attributes()
    attr = Dict{Tuple{Int32, Int32}, Int32}()
    for i in 0:(ndevices()-1)
        d = CuDevice(i)
        attr[(
            attribute(d, CUDA.CU_DEVICE_ATTRIBUTE_PCI_BUS_ID),
            attribute(d, CUDA.CU_DEVICE_ATTRIBUTE_PCI_DEVICE_ID)
        )] = d
    end
    attr
end

get_device_attributes (generic function with 1 method)

In [9]:
using Hwloc, AbstractTrees


import AbstractTrees: PreOrderDFS
import Hwloc: hwloc_pci_class_string

In [34]:
function tag_subtree!(tree_node, val)
    for n in collect(AbstractTrees.PreOrderDFS(tree_node))
        n.tag = val
    end
end

function distance_to_core!(node, target_index)
    # shield re-entrance when iterating
    node.tag = 1

    if node.type == :PU
        # println("Checking: $(nodevalue(node).os_index)")
        if nodevalue(node).os_index == target_index
            return true, 0
        end
    end

    for child in node.children
        if child.tag == 1
            continue
        end

        found, dist = distance_to_core!(child, target_index)
        if found
            return true, dist + 1
        end
    end

    if node.parent != nothing
        found, dist = distance_to_core!(node.parent, target_index)
        if found
            return true, dist + 1
        end
    end

    return false, typemax(Int)
end

function distance_to_core(root, node, target_index)
    tag_subtree!(root, 0) 
    found, dist = distance_to_core!(node, target_index)
    tag_subtree!(root, 0) 
    return found, dist
end

distance_to_core (generic function with 1 method)

In [35]:
get_device_attributes()

Dict{Tuple{Int32, Int32}, Int32} with 4 entries:
  (65, 0)  => 1
  (193, 0) => 3
  (130, 0) => 2
  (3, 0)   => 0

In [36]:
sys_devs = children(gettopology())
pci_devs = PreOrderDFS(sys_devs) |> collect |> filter(x->x.type==:PCI_Device)
gpu_devs = pci_devs |> filter(x->hwloc_pci_class_string(nodevalue(x).attr.class_id) == "3D")

function get_device_distances(core)
    attr = get_device_attributes()
    dist = Dict{Int32, Int32}()
    dev  = Dict{Int32, Int32}()
    for d in gpu_devs
        idx = attr[(nodevalue(d).attr.bus, nodevalue(d).attr.dev)]
        found, dev_d = distance_to_core(sys_devs, d, core)
        if found
            dist[idx] = dev_d
            dev[dev_d] = idx
        end
    end
    dist, dev
end

get_device_distances (generic function with 1 method)

In [37]:
cpucycle_coreid()

49

In [48]:
dist, dev = get_device_distances(cpucycle_coreid())
closest_dev = dev[dev |> keys |> minimum]

0

In [45]:
dist

Dict{Int32, Int32} with 4 entries:
  0 => 18
  2 => 516
  3 => 516
  1 => 516

In [46]:
dev

Dict{Int32, Int32} with 2 entries:
  18  => 0
  516 => 1

0